# Plant Pathology 2020 with fastai2

This is a fork of Rony K Roy & Soly James notebook. Link: [https://www.kaggle.com/ronyroy/effnet-fastai-folds-x5](https://www.kaggle.com/ronyroy/effnet-fastai-folds-x5)
and
inspired by [lex's notebook here](https://www.kaggle.com/lextoumbourou/plant-pathology-2020-eda-training-fastai2)  

This ia a starter example for classifying categories of foliar diseases in apple trees.

This ought to help one exlore EfficientNets in Pytorch using GPUs

In [ ]:
!pip install git+https://github.com/fastai/fastcore > /dev/null
!pip install git+https://github.com/fastai/fastai2 > /dev/null
!pip install iterative-stratification > /dev/null

In [ ]:
!pip install efficientnet-pytorch

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
from collections import Counter
from pathlib import Path

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from tqdm.notebook import tqdm
from torchvision.models import densenet121
from sklearn.metrics import roc_auc_score
from torch.utils.data.sampler import WeightedRandomSampler

from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
from efficientnet_pytorch import EfficientNet

## Params

In [ ]:
DATA_PATH = Path('../input/plant-pathology-2020-fgvc7')
IMG_PATH = DATA_PATH / 'images'
LABEL_COLS = ['healthy', 'multiple_diseases', 'rust', 'scab']

IMG_SIZE = 512
SEED = 420
N_FOLDS = 5
BS = 8#16
N_FOLDS = 5

ARCH = densenet121

In [ ]:
seed_everything(SEED)

## EDA

In [ ]:
train_df = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
test_df = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')

In [ ]:
train_df.head()

### Dataset size

In [ ]:
(len(train_df), len(test_df))

### Label distribution

In [ ]:
_, axes = plt.subplots(ncols=4, nrows=1, constrained_layout=True, figsize=(10, 3))
for ax, column in zip(axes, LABEL_COLS):
    train_df[column].value_counts().plot.bar(title=column, ax=ax)
plt.show()

Let's see how many times the labels appear together.

In [ ]:
train_df.iloc[:,1:-1].sum(axis=1).value_counts()

In [ ]:
train_df[['healthy', 'multiple_diseases', 'rust', 'scab']].sum(axis=1).unique()

Looks like never. So this appears to be multiclass but not multilabel classification.

In [ ]:
test_df.head()

### Img size distribution

In [ ]:
hs, ws = [], []
for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    img = Image.open(IMG_PATH/(row.image_id+'.jpg'))
    h, w = img.size
    hs.append(h)
    ws.append(w)

In [ ]:
set(hs), set(ws)

In [ ]:
_, axes = plt.subplots(ncols=2, nrows=1, constrained_layout=True, figsize=(10, 3))
for ax, column, vals in zip(axes, ['heights', 'widths'], [hs, ws]):
    ax.hist(vals, bins=100)
    ax.set_title(f'{column} hist')

plt.show()

In [ ]:
Counter(hs), Counter(ws)

Vast majority of images are big: 2048x1819.

### Colour distribution

In [ ]:
red_values = []; green_values = []; blue_values = []; all_channels = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    img = np.array(Image.open(IMG_PATH/(row.image_id+'.jpg')))
    red_values.append(np.mean(img[:, :, 0]))
    green_values.append(np.mean(img[:, :, 1]))
    blue_values.append(np.mean(img[:, :, 2]))
    all_channels.append(np.mean(img))

In [ ]:
_, axes = plt.subplots(ncols=4, nrows=1, constrained_layout=True, figsize=(16, 3), sharey=True)
for ax, column, vals, c in zip(
    axes,
    ['red', 'green', 'blue', 'all colours'],
    [red_values, green_values, blue_values, all_channels],
    'rgbk'
):
    ax.hist(vals, bins=100, color=c)
    ax.set_title(f'{column} hist')

plt.show()

## Create folds

I'll use iterative stratification to create balanced folds.

In [ ]:
train_df['fold'] = -1

strat_kfold = MultilabelStratifiedKFold(n_splits=N_FOLDS, random_state=SEED, shuffle=True)
for i, (_, test_index) in enumerate(strat_kfold.split(train_df.image_id.values, train_df.iloc[:,1:].values)):
    train_df.iloc[test_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')

In [ ]:
train_df.fold.value_counts().plot.bar()

In [ ]:
train_df.to_csv('train_with_strat_folds.csv', index=False)

## Data (inc oversampling)

I'm not sure how to load one-hot encoded labels into fastai2, so going to convert to a string label.

In [ ]:
def get_label(row):
    for k, v in row[LABEL_COLS].items():
        if v == 1:
            return k

In [ ]:
train_df['label'] = train_df.apply(get_label, axis=1)

Right now, I'm x2 the number of multiple diseases labels as that appears to be the majorly unrepresented class.

In [ ]:
def get_data(fold):
    train_df_no_val = train_df.query(f'fold != {fold}')
    train_df_just_val = train_df.query(f'fold == {fold}')

    train_df_bal = pd.concat(
        [train_df_no_val.query('label != "multiple_diseases"'), train_df_just_val] +
        [train_df_no_val.query('label == "multiple_diseases"')] * 4 # back to 4 as this was hs
    ).sample(frac=1.0, random_state=SEED).reset_index(drop=True)

    datablock = DataBlock(
        blocks=(ImageBlock, CategoryBlock(vocab=LABEL_COLS)),
        getters=[
            ColReader('image_id', pref=IMG_PATH, suff='.jpg'),
            ColReader('label')
        ],
        splitter=IndexSplitter(train_df_bal.loc[train_df_bal.fold==fold].index),
        item_tfms=Resize(IMG_SIZE),
        batch_tfms=aug_transforms(size=IMG_SIZE, max_rotate=30., min_scale=0.75, flip_vert=True, do_flip=True)
    )
    return datablock.dataloaders(source=train_df_bal, bs=BS)

In [ ]:
def get_data_larger(fold):
    train_df_no_val = train_df.query(f'fold != {fold}')
    train_df_just_val = train_df.query(f'fold == {fold}')

    train_df_bal = pd.concat(
        [train_df_no_val.query('label != "multiple_diseases"'), train_df_just_val] +
        [train_df_no_val.query('label == "multiple_diseases"')] * 4 # back to 4 as this was hs
    ).sample(frac=1.0, random_state=SEED).reset_index(drop=True)

    datablock = DataBlock(
        blocks=(ImageBlock, CategoryBlock(vocab=LABEL_COLS)),
        getters=[
            ColReader('image_id', pref=IMG_PATH, suff='.jpg'),
            ColReader('label')
        ],
        splitter=IndexSplitter(train_df_bal.loc[train_df_bal.fold==fold].index),
        item_tfms=Resize(IMG_SIZE*2),
        batch_tfms=aug_transforms(size=IMG_SIZE*2, max_rotate=30., min_scale=0.75, flip_vert=True, do_flip=True)
    )
    return datablock.dataloaders(source=train_df_bal, bs=BS/2)

In [ ]:
dls = get_data(fold=0)

In [ ]:
dls.c

In [ ]:
dls.show_batch()

## Training

In [ ]:
def comp_metric(preds, targs, labels=range(len(LABEL_COLS))):
    # One-hot encode targets
    targs = np.eye(4)[targs]
    return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

def healthy_roc_auc(*args):
    return comp_metric(*args, labels=[0])

def multiple_diseases_roc_auc(*args):
    return comp_metric(*args, labels=[1])

def rust_roc_auc(*args):
    return comp_metric(*args, labels=[2])

def scab_roc_auc(*args):
    return comp_metric(*args, labels=[3])

# efficientnet-b7

focus here is on efficientnet-b7
Please try with other efficientnet as well

In [ ]:
def get_learner(fold_num, lr=1e-3):
    opt_func = partial(Adam, lr=lr, wd=0.01, eps=1e-8)

    data = get_data(fold_num)
    
    model = EfficientNet.from_pretrained("efficientnet-b7", advprop=True)
    #model = EfficientNet.from_name("efficientnet-b7")
    #model = EfficientNet.from_pretrained("efficientnet-b8", advprop=True) # weights run to NaN
    #model = EfficientNet.from_name('efficientnet-b4') 
    #model._fc = nn.Linear(1280, data.c)# the last layer... # works for b0,b1
    #model._fc = nn.Linear(1536, data.c)# the last layer... B3
    #model._fc = nn.Linear(1792, data.c)# the last layer... B4
    #model._fc = nn.Linear(2048, data.c)# the last layer... B5
    #model._fc = nn.Linear(2304, data.c)# the last layer... B6
    model._fc = nn.Linear(2560, data.c)# the last layer... B7
    #model._fc = nn.Linear(2816, data.c)# the last layer... B8

    learn = Learner(
        dls, model, opt_func=opt_func,
        loss_func=LabelSmoothingCrossEntropy(),
        #callback_fns = [partial(OverSamplingCallback)],  
        metrics=[
            AccumMetric(healthy_roc_auc, flatten=False),
            AccumMetric(multiple_diseases_roc_auc, flatten=False),
            AccumMetric(rust_roc_auc, flatten=False),
            AccumMetric(scab_roc_auc, flatten=False),
            AccumMetric(comp_metric, flatten=False)]
        ).to_fp16()
    return learn

In [ ]:
get_learner(fold_num=0).lr_find()

In [ ]:
def print_metrics(val_preds, val_labels):
    comp_metric_fold = comp_metric(val_preds, val_labels)
    print(f'Comp metric: {comp_metric_fold}')
    
    healthy_roc_auc_metric = healthy_roc_auc(val_preds, val_labels)
    print(f'Healthy metric: {healthy_roc_auc_metric}')
    
    multiple_diseases_roc_auc_metric = multiple_diseases_roc_auc(val_preds, val_labels)
    print(f'Multi disease: {multiple_diseases_roc_auc_metric}')
    
    rust_roc_auc_metric = rust_roc_auc(val_preds, val_labels)
    print(f'Rust metric: {rust_roc_auc_metric}')
    
    scab_roc_auc_metric = scab_roc_auc(val_preds, val_labels)
    print(f'Scab metric: {scab_roc_auc_metric}')

In [ ]:
all_val_preds = []
all_val_labels = []
all_test_preds = []

for i in range(N_FOLDS):
    print(f'Fold {i} results')

    learn = get_learner(fold_num=i)

    #learn.fit_one_cycle(5)
    learn.fit_one_cycle(1)
    learn.unfreeze()

    #learn.fit_one_cycle(6, slice(1e-5, 1e-4))
    learn.fit_one_cycle(1, slice(1e-5, 1e-4))
    
    learn.recorder.plot_loss()
    
    learn.save(f'model_fold_{i}')
    
    learn.freeze()
    
    learn.data = get_data_larger(i)
    
    lr_min,_ = learn.lr_find()
    
    #learn.fit_one_cycle(5, slice(lr_min))
    learn.fit_one_cycle(1, slice(lr_min))
    
    val_preds, val_labels = learn.get_preds()
    
    print_metrics(val_preds, val_labels)
    
    all_val_preds.append(val_preds)
    all_val_labels.append(val_labels)
    
    test_dl = dls.test_dl(test_df)
    test_preds, _ = learn.get_preds(dl=test_dl)
    all_test_preds.append(test_preds)
    
plt.show()

In [ ]:
print_metrics(np.concatenate(all_val_preds), np.concatenate(all_val_labels))

## Interpret

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15, 10))

In [ ]:
interp.plot_confusion_matrix(normalize=True, figsize=(6, 6))

## Test predictions

In [ ]:
test_df_output = pd.concat([test_df, pd.DataFrame(np.mean(np.stack(all_test_preds), axis=0), columns=LABEL_COLS)], axis=1)

In [ ]:
test_df_output.head()

In [ ]:
test_df_output.to_csv('submission.csv', index=False)

In [ ]:
!head -n 5 submission.csv